Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/using-mlflow/train-deploy-pytorch/train-deploy-pytorch.png)

## Use MLflow with Azure Machine Learning to Train and Deploy PyTorch Image Classifier

This example shows you how to use MLflow together with Azure Machine Learning services for tracking the metrics and artifacts while training a PyTorch model to classify MNIST digit images, and then deploy the model  as a web service. You'll learn how to:

 1. Set up MLflow tracking URI so as to use Azure ML
 2. Create experiment
 3. Instrument your model with MLflow tracking
 4. Train a PyTorch model locally
 5. Train a model on GPU compute on Azure
 6. View your experiment within your Azure ML Workspace in Azure Portal
 7. Create a Docker image from the trained model
 8. Deploy the model as a web service on Azure Container Instance
 9. Call the model to make predictions
 
### Pre-requisites
 
Make sure you have completed the [Configuration](../../../configuration.ipnyb) notebook to set up your Azure Machine Learning workspace and ensure other common prerequisites are met.

Also, install mlflow-azureml package using ```pip install mlflow-azureml```. Note that mlflow-azureml installs mlflow package itself as a dependency, if you haven't done so previously.

### Set-up

Import packages and check versions of Azure ML SDK and MLflow installed on your computer. Then connect to your Workspace.

In [ ]:
import sys, os
import mlflow
import mlflow.azureml
import mlflow.sklearn

import azureml.core
from azureml.core import Workspace


print("SDK version:", azureml.core.VERSION)
print("MLflow version:", mlflow.version.VERSION)

In [ ]:
ws = Workspace.from_config()
ws.get_details()

### Set tracking URI

Set the MLFlow tracking URI to point to your Azure ML Workspace. The subsequent logging calls from MLFlow APIs will go to Azure ML services and will be tracked under your Workspace.

In [ ]:
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

### Create Experiment

In both MLflow and Azure ML, training runs are grouped into experiments. Let's create one for our experimentation.

In [ ]:
experiment_name = "pytorch-with-mlflow"
mlflow.set_experiment(experiment_name)

### Train model locally while logging metrics and artifacts

The ```scripts/train.py``` program contains the code to load the image dataset, and train and test the model. Within this program, the train.driver function wraps the end-to-end workflow.

Within the driver, the ```mlflow.start_run``` starts MLflow tracking. Then, ```mlflow.log_metric``` functions are used to track the convergence of the neural network training iterations. Finally ```mlflow.pytorch.save_model``` is used to save the trained model in framework-aware manner.

Let's add the program to search path, import it as a module, and then invoke the driver function. Note that the training can take few minutes.

In [ ]:
lib_path = os.path.abspath("scripts")
sys.path.append(lib_path)

import train

In [ ]:
run = train.driver()

You can view the metrics of the run at Azure Portal

In [ ]:
print(azureml.mlflow.get_portal_url(run))

### Train model on GPU compute on Azure

Next, let's run the same script on GPU-enabled compute for faster training. If you've completed the the [Configuration](../../../configuration.ipnyb) notebook, you should have a GPU cluster named "gpu-cluster" available in your workspace. Otherwise, follow the instructions in the notebook to create one. For simplicity, this example uses single process on single VM to train the model.

Create a PyTorch estimator to specify the training configuration: script, compute as well as additional packages needed. To enable MLflow tracking, include ```azureml-mlflow``` as pip package. The low-level specifications for the training run are encapsulated in the estimator instance.

In [ ]:
from azureml.train.dnn import PyTorch

pt = PyTorch(source_directory="./scripts", 
             entry_script = "train.py", 
             compute_target = "gpu-cluster", 
             node_count = 1, 
             process_count_per_node = 1, 
             use_gpu=True,
             pip_packages = ["azureml-mlflow", "Pillow==6.0.0"])



Get a reference to the experiment you created previously, but this time, as Azure Machine Learning experiment object.

Then, use ```Experiment.submit``` method to start the remote training run. Note that the first training run often takes longer as Azure Machine Learning service builds the Docker image for executing the script. Subsequent runs will be faster as cached image is used.

In [ ]:
from azureml.core import Experiment

exp = Experiment(ws, experiment_name)
run = exp.submit(pt)

You can monitor the run and its metrics on Azure Portal.

In [ ]:
run

Also, you can wait for run to complete.

In [ ]:
run.wait_for_completion(show_output=True)

### Deploy model as web service

To deploy a web service, first create a Docker image, and then deploy that Docker image on inferencing compute.

The ```mlflow.azureml.build_image``` function builds a Docker image from saved PyTorch model in a framework-aware manner. It automatically creates the PyTorch-specific inferencing wrapper code and specififies package dependencies for you.

In [ ]:
run.get_file_names()

Then build a docker image using *runs:/&lt;run.id&gt;/model* as the model_uri path.

Note that the image building can take several minutes.

In [ ]:
model_path = "model"


azure_image, azure_model = mlflow.azureml.build_image(model_uri='runs:/{}/{}'.format(run.id, model_path),
                                                      workspace=ws,
                                                      model_name='pytorch_mnist',
                                                      image_name='pytorch-mnist-img',
                                                      synchronous=True)

Then, deploy the Docker image to Azure Container Instance: a serverless compute capable of running a single container. You can tag and add descriptions to help keep track of your web service. 

[Other inferencing compute choices](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where) include Azure Kubernetes Service which provides scalable endpoint suitable for production use.

Note that the service deployment can take several minutes.

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, 
                                                memory_gb=5, 
                                                tags={"data": "MNIST",  "method" : "pytorch"}, 
                                                description="Predict using webservice")


# Deploy the image to Azure Container Instances (ACI) for real-time serving
webservice = Webservice.deploy_from_image(
    image=azure_image, workspace=ws, name="pytorch-mnist-1", deployment_config=aci_config)


webservice.wait_for_deployment()

Once the deployment has completed you can check the scoring URI of the web service.

In [ ]:
print("Scoring URI is: {}".format(webservice.scoring_uri))

In case of a service creation issue, you can use ```webservice.get_logs()``` to get logs to debug.

### Make predictions using web service

To make the web service, create a test data set as normalized PyTorch tensors. 

Then, let's define a utility function that takes a random image and converts it into format and shape suitable for as input to PyTorch inferencing end-point. The conversion is done by: 

 1. Select a random (image, label) tuple
 2. Take the image and converting the tensor to NumPy array 
 3. Reshape array into 1 x 1 x N array
    * 1 image in batch, 1 color channel, N = 784 pixels for MNIST images
    * Note also ```x = x.view(-1, 1, 28, 28)``` in net definition in ```train.py``` program to shape incoming scoring requests.
 4. Convert the NumPy array to list to make it into a built-in type.
 5. Create a dictionary {"data", &lt;list&gt;} that can be converted to JSON string for web service requests.

In [ ]:
from torchvision import datasets, transforms
import random
import numpy as np

test_data = datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))]))


def get_random_image():
    image_idx = random.randint(0,len(test_data))
    image_as_tensor = test_data[image_idx][0]
    return {"data": elem for elem in image_as_tensor.numpy().reshape(1,1,-1).tolist()}

Then, invoke the web service using a random test image. Convert the dictionary containing the image to JSON string before passing it to web service.

The response contains the raw scores for each label, with greater value indicating higher probability. Sort the labels and select the one with greatest score to get the prediction. Let's also plot the image sent to web service for comparison purposes.

In [ ]:
%matplotlib inline

import json
import matplotlib.pyplot as plt

test_image = get_random_image()

response = webservice.run(json.dumps(test_image))

response = sorted(response[0].items(), key = lambda x: x[1], reverse = True)


print("Predicted label:", response[0][0])
plt.imshow(np.array(test_image["data"]).reshape(28,28), cmap = "gray")

You can also call the web service using a raw POST method against the web service

In [ ]:
import requests

response = requests.post(url=webservice.scoring_uri, data=json.dumps(test_image),headers={"Content-type": "application/json"})
print(response.text)